<a href="https://colab.research.google.com/github/btshow/ARMA_Simulator/blob/main/Colab_only_SmartPowerPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!apt-get install python3.6
!python --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.6 is already the newest version (3.6.9-1~18.04ubuntu1.3).
python3.6 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Python 3.6.9


In [10]:
!git clone https://github.com/GoogleCloudPlatform/professional-services.git
!pip install pandas==0.23.4
!pip install scikit-learn==0.20.2
%tensorflow_version 1.14.0
import pandas as pd
from datetime import datetime
import argparse
import io
import json
import os
import pandas as pd
import numpy as np
import sklearn.metrics
#import trainer.model as model
import tensorflow as tf
from google.cloud import storage
from tensorflow import keras

print('tf version is', tf.__version__)
print('pandas version is', pd.__version__)

tf.enable_eager_execution()

fatal: destination path 'professional-services' already exists and is not an empty directory.
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
tf version is 1.15.2
pandas version is 0.23.4


In [ ]:

'''
#official requirements
REQUIRED_PACKAGES = [
  "tensorflow==1.12.0",
  "google-cloud-storage==1.13.2",
  "pandas==0.23.4",
  "scikit-learn==0.20.2",
]
#pythonVersion: "3.5"
'''

'\n#official requirements\nREQUIRED_PACKAGES = [\n  "tensorflow==1.12.0",\n  "google-cloud-storage==1.13.2",\n  "pandas==0.23.4",\n  "scikit-learn==0.20.2",\n]\n\n#pythonVersion: "3.5"\n'

# Access to Google Drive

In [11]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Load csv file from Google Drive into Colab VM and as a pandas df

In [12]:

def load_csv(link, name):
  '''
   returns a pandas df corresponding to a csv loaded from google drive 
   as a side effect loads a csv file named <name> into colab VM
  link: string, containing a google drive shareable link (obtained from google drive right-click on file)
    e.g. 'https://drive.google.com/file/d/1GNR378iZTsGjiWwJxvb_sCug8IDVcDSS/view?usp=sharing'
  name: string, containing how the new csv file should be named after loading into
   the colab VM 
    e.g. 'eval.csv'
'''
  fluff, id = link.split('d/')
  id, fluff = id.split('/view')
  downloaded = drive.CreateFile({'id':id}) 
  return downloaded.GetContentFile(name), pd.read_csv(name)  

# eval.csv
_, df_eval =load_csv(
    'https://drive.google.com/file/d/1GNR378iZTsGjiWwJxvb_sCug8IDVcDSS/view?usp=sharing',
    'eval.csv'
  )

# train.csv
_, df_train =load_csv(
    'https://drive.google.com/file/d/1OobRyJFXWDSF4MOk4bnloYKF-UVJJuyj/view?usp=sharing',
    'train.csv'
)

In [13]:
pd.set_option('max_columns', None)
print(df_train.head())
print(df_train.columns)

                  time  running_machine  laptop2  washing_machine  \
0  2013-06-01 00:00:01                1        0                3   
1  2013-06-01 00:00:07                1        0                3   
2  2013-06-01 00:00:13                1        0                3   
3  2013-06-01 00:00:19                1        0                3   
4  2013-06-01 00:00:25                1        0                3   

   dish_washer  fridge  microwave  toaster  playstation  modem  cooker  \
0            0      84          0        1            0      9       0   
1            1      83          0        1            0      9       0   
2            1      83          0        1            0      9       0   
3            1      84          0        1            0      9       0   
4            0      83          0        1            0      9       0   

   laptop  monitor  speakers  server  router  server_hdd  kettle  rice_cooker  \
0      15       61        10      13       6           0   

# Specify Data Iterator Functions

In [14]:
def json_serving_input_fn(feat_names):
  """
  Build the serving inputs

  Args:
    feat_name   - list, Name list of features used in the prediction model.

  Returns:
    tf.estimator.export.ServingInputReceive
  """

  def serving_input_fn():
    feat_cols = [tf.feature_column.numeric_column(x) for x in feat_names]
    inputs = {}
    for feat in feat_cols:
      inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

  return serving_input_fn


def make_input_fn(data_file,
                  seq_len,
                  batch_size,
                  cols=None,
                  num_epochs=None,
                  shuffle=False,
                  train_flag=False,
                  filter_prob=1.0):
  """Input function for estimator.

  Input function for estimator.

  Args:
    data_file   - string, Path to input csv file.
    seq_len     - int, Length of time sequence.
    batch_size  - int, Mini-batch size.
    num_epochs  - int, Number of epochs
    shuffle     - bool, Whether to shuffle the data
    cols        - list, Columns to extract from csv file.
    train_flag  - bool, Whether in the training phase, we want to
                  ignore sequences when all appliances are off.
    filter_prob - float, The probability to pass data sequences with all
                  appliances being 'off', only valid when train_flag is True.
  Returns:
    tf.data.Iterator.
  """

  def _mk_data(*argv):
    """Format data for further processing.

    This function slices data into subsequences, extracts the flags
    from the last time steps and treat each as the target for the subsequences.
    """
    data = {'ActivePower_{}'.format(i + 1): x
            for i, x in enumerate(tf.split(argv[0], seq_len))}
    # Only take the label of the last time step in the sequence as target
    flags = [tf.split(x, seq_len)[-1][0] for x in argv[1:]]

    #return 2 elements: 
    # data i.e. the gross electricty consumption series as well as
    # a tensor for each input sequence with the labels stacked into one 1-d tensor
    # as well as cast into the unit8 dtype format
    return data, tf.cast(tf.stack(flags), dtype=tf.uint8)

  def _filter_data(data, labels):
    """Filter those sequences with all appliances 'off'.

    Filter those sequences with all appliances 'off'.
    However, with filter_prob we pass the sequence.
    """
    rand_num = tf.random_uniform([], 0, 1, dtype=tf.float64)
    thresh = tf.constant(filter_prob, dtype=tf.float64, shape=[])
    is_all_zero = tf.equal(tf.reduce_sum(labels), 0)
    return tf.logical_or(tf.logical_not(is_all_zero),
                         tf.less(rand_num, thresh))
  ## 
  '''
  From here down: A iterator is created that can be used to feed training data
  iteratively to the training algorithm during model training
  The iterator imposes the data formats provided in record_defaults on the csv
  file read in subsequently
  The iterator uses a sliding window of size seq_len to split the time series in 
  several overlapping! time series of each size seq_len and feeds these 
  new time series as an iterator i.e. as a DatasetV1Adapter Object
  Using _mk_data, the iterator splits each sequence into a 1d tensor of 
  [seq length] elements a and 1d tensor of 0-1 labels for each sequence 
  (corresponding to the on-/off status off the appliance during the last 
  sequence element over all appliances)

  The filter function is optional and only useful when we want to reduce the 
  number of sequences with all appliances turned of

  The other function modify the DatasetV1Adapter Object in order 
  to shuffle the whole set of sequences before using it further, 
  to repeat the full batch a number of epochs
  to define a size for each minibatch
  Finally an tensorflow.python.data.ops.iterator_ops.IteratorV2 iterator object
  is created using the DatasetV1Adapter Object dataset.
  The iterator object can now be used to feed an ML model 


  IMPORTANT: To create an iterator for a gross electricty consumption forecasting 
  model, _mk_data() has to be modified - particularly the line 
  flags = [tf.split(x, seq_len)[-1][0] for x in argv[1:]]
  here has to modified, such that the last element of the electricity consumption 
  series is used as target variable instead of argv[1:]
  Hence, this would be worth a try to generate the forecasting target variable
  flags = [tf.split(x, seq_len)[-1][0] for x in argv[0]]

  
  def _mk_data(*argv):
    """Format data for further processing.

    This function slices data into subsequences, extracts the flags
    from the last time steps and treat each as the target for the subsequences.
    """
    data = {'ActivePower_{}'.format(i + 1): x
            for i, x in enumerate(tf.split(argv[0], seq_len))}
    # Only take the label of the last time step in the sequence as target
    flags = [tf.split(x, seq_len)[-1][0] for x in argv[1:]]
    return data, tf.cast(tf.stack(flags), dtype=tf.uint8)  
  '''
  
  # create the list record_defaults (a list of tf DType objects e.g. tf.float64)
  record_defaults = [tf.float64, ] + [tf.int32] * (len(cols) - 1)

  #create the  'tensorflow.contrib.data.python.ops.readers.CsvDataset'-object dataset
  dataset = tf.contrib.data.CsvDataset([data_file, ],
                                       record_defaults,
                                       header=True,
                                       select_cols=cols)
  # create the 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'-object dataset
  dataset = dataset.apply(
    tf.contrib.data.sliding_window_batch(window_size=seq_len))
  
  # applies _mk_data to each element of dataset i.e. to each sliding
  dataset = dataset.map(_mk_data)

  if train_flag:
    dataset = dataset.filter(_filter_data).shuffle(60 * 60 * 24 * 7)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=batch_size * 10)

  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=1)

  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next()



# Specify ML Model and its Outputs


  Denote gross energy in the house as a sequence
  $(x_t, x_{t+1}, \cdots, x_{t+n-1}) \in \mathcal{R}^n$

  Denote the on/off states of appliances at time $t$ as
  $y_{t} = \{y^i_t \mid y^i_t = [{appliance}\ i\ is\ on\ at\ time\ t ]\}$,
  
  We are learning the function
  $f(x_t, x_{t+1}, \cdots, x_{t+n-1}) \mapsto \hat{y}_{t+n-1}$.

In [15]:
def model_fn(features, labels, mode, params):
  """Build a customized model for energy disaggregation.

  The model authoring uses pure tensorflow.layers.

  Args:
    features: dict(str, tf.data.Dataset)
    labels: tf.data.Dataset
    mode: One of {tf.estimator.ModeKeys.EVAL,
                  tf.estimator.ModeKeys.TRAIN,
                  tf.estimator.ModeKeys.PREDICT}
    params: Other related parameters

  Returns:
    tf.estimator.EstimatorSpec.
  """

  if mode == tf.estimator.ModeKeys.TRAIN:
    tf.logging.info('TRAIN')
  else:
    tf.logging.info('EVAL | PREDICT')

  feat_cols = [tf.feature_column.numeric_column(x) for x in params['feat_cols']]
  seq_data = tf.feature_column.input_layer(features, feat_cols)

  if not params['use_keras']:
    tf.logging.info('Tensorflow authoring')

    seq_data_shape = tf.shape(seq_data)
    batch_size = seq_data_shape[0]

    # RNN network using multilayer LSTM

    #dropout in LSTM layer with 'lstm_size' units
    cells = [tf.nn.rnn_cell.DropoutWrapper(
      tf.nn.rnn_cell.LSTMCell(params['lstm_size']), input_keep_prob=1 - params['dropout_rate'])
      for _ in range(params['num_layers'])]

    #  Compose lstm (ie "RNN") cell composed sequentially of multiple cells given by "cells"
    lstm = tf.nn.rnn_cell.MultiRNNCell(cells)

    # Initialize the state of each LSTM cell to zero
    state = lstm.zero_state(batch_size, dtype=tf.float32)

    # Unroll multiple time steps and the output size is:
    # [batch_size, max_time, cell.output_size]
    outputs, states = tf.nn.dynamic_rnn(cell=lstm,
                                        inputs=tf.expand_dims(seq_data, -1),
                                        initial_state=state,
                                        dtype=tf.float32)

    # Flatten the 3D output to 2D as [batch_size, max_time * cell.output_size]
    flatten_outputs = tf.layers.Flatten()(outputs)

    # A fully connected layer. The number of output equals the number of target appliances
    logits = tf.layers.Dense(params['num_appliances'])(flatten_outputs)

  else:
    tf.logging.info('Keras authoring')

####### RNN Model code zu  sequence2sequence anpassen #######
    # RNN network using multilayer LSTM with the help of Keras
    model = keras.Sequential()
    for _ in range(params['num_layers']):
      model.add(
        keras.layers.LSTM(params['lstm_size'],
                          dropout=params['dropout_rate'],
                          return_sequences=True)
      )

    # Flatten the 3D output to 2D as [batch_size, max_time * cell.output_size]
    model.add(keras.layers.Flatten())
    # A fully connected layer. The number of output equals the number of target appliances
    model.add(keras.layers.Dense(params['num_appliances']))


    # Logits can be easily computed using Keras functional API
    logits = model(tf.expand_dims(seq_data, -1))

####### RNN Output code zu sequence2sequence anpassen (nicht probs) #######
  # Probability of turning-on of each appliances corresponding output are computed by applying a sigmoid function on logits
  probs = tf.nn.sigmoid(logits)
  predictions = {
    'probabilities': probs,
    'logits': logits
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)

####### Loss function code zu sequence2sequence loss anpassen (nicht sigmoid_crossent) #######
  # Binary cross entropy is used as loss function
  loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels, logits=logits)
  loss_avg = tf.reduce_mean(loss) # kann hier mean loss bleiben. wenn nicht muss loss_avg  evtl. bei train_op auch angepasst werden

####### Predictions code zu sequence2sequence anpassen (nicht predicted classes) #######
  # Predicted Classses
  predicted_classes = tf.cast(tf.round(probs), tf.uint8)

####### Metrics code zu sequence2sequence anpassen (nicht precision usw) #######
  #Evaluation Metrics
  precision = tf.metrics.precision(labels=labels,
                                   predictions=predicted_classes)
  
  recall = tf.metrics.recall(labels=labels,
                             predictions=predicted_classes)
  f1_score = tf.contrib.metrics.f1_score(labels=labels,
                                         predictions=predicted_classes)

  metrics = {'precision': precision,
             'recall': recall,
             'f_measure': f1_score}


  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

  optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
  train_op = optimizer.minimize(loss_avg, global_step=tf.train.get_global_step()) # hier evtl anpassen wenn nicht loss_avg
  return tf.estimator.EstimatorSpec(mode,
                                    loss=loss,
                                    train_op=train_op)

## Specify Model Hyperparameters and Training Specification

In [20]:
if __name__ == '__main__':

  ''' 
  Set Arguments to be passed to the ML model

  MAIN OUTPUT from here and used in "ML Experiment" is  args

  args contains all hyperparamter and model training specifications

  After this cell you can run the 
  '''

  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
    '--train-file',
    help='GCS or local paths to training data',
    default='train.csv'
  )
  parser.add_argument(
    '--num-epochs',
    help="""
      Maximum number of training data epochs on which to train.
      If both --max-steps and --num-epochs are specified,
      the training job will run for --max-steps or --num-epochs,
      whichever occurs first. If unspecified will run for --max-steps.
      """,
    type=int,
    default=2 #originally = 40
  )
  parser.add_argument(
    '--train-batch-size',
    help='Batch size for training steps',
    type=int,
    default=64
  )
  parser.add_argument(
    '--eval-file',
    help='GCS or local paths to evaluation data',
    default='eval.csv'
  )
  parser.add_argument(
    '--eval-batch-size',
    help='Batch size for evaluation steps',
    type=int,
    default=64
  )
  parser.add_argument(
    '--test-file',
    help='GCS or local paths to test data',
  )
  # Training arguments
  parser.add_argument(
    '--seq-len',
    help='Length of cropped sequence',
    default=5,
    type=int
  )
  parser.add_argument(
    '--lstm-size',
    help='Size of lstm',
    default=5, #originally = 131
    type=int
  )
  parser.add_argument(
    '--num-layers',
    help='Number of layers in the model',
    default=1, #originally = 4
    type=int
  )
  parser.add_argument(
    '--dropout-rate',
    help='The rate of drop out',
    default=0.3204,
    type=float
  )
  parser.add_argument(
    '--learning-rate',
    help='Learning rate',
    default=8.1729e-5,
    type=float
  )
  parser.add_argument(
    '--filter-prob',
    help='Filter probability',
    default=0.6827,
    type=float
  )
  parser.add_argument(
    '--job-dir',
    help='GCS location to write checkpoints and export models',
    default='/tmp/estimator'
  )
  parser.add_argument(
    '--model-name',
    help='name of the model',
    default='estimator'
  )
  # Argument to turn on all logging
  parser.add_argument(
    '--verbosity',
    choices=[
      'DEBUG',
      'ERROR',
      'FATAL',
      'INFO',
      'WARN'
    ],
    default='INFO',
  )
  # Experiment arguments
  parser.add_argument(
    '--train-steps',
    help="""\
      Steps to run the training job for. If --num-epochs is not specified,
      this must be. Otherwise the training job will run indefinitely.\
      """,
    default=1e3, #originally = 1e5
    type=int
  )
  parser.add_argument(
    '--eval-steps',
    help='Number of steps to run evalution for at each checkpoint',
    default=1e2, #originally = 1e3
    type=int
  )
  parser.add_argument(
    '--test',
    help='Whether to test a model',
    default=False,
    type=bool
  )
  parser.add_argument(
    '--keras',
    help='Whether use keras authoring',
    action='store_true'
  )

  args, _ = parser.parse_known_args()

  # Set python level verbosity
  tf.logging.set_verbosity(args.verbosity)
  # Set C++ Graph Execution level verbosity
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(tf.logging.__dict__[args.verbosity] / 10)

  config = args.__dict__
  for k, v in config.items():
    tf.logging.info('{}: {}'.format(k, v))
    
 

INFO:tensorflow:train_file: train.csv
INFO:tensorflow:num_epochs: 2
INFO:tensorflow:train_batch_size: 64
INFO:tensorflow:eval_file: eval.csv
INFO:tensorflow:eval_batch_size: 64
INFO:tensorflow:test_file: None
INFO:tensorflow:seq_len: 5
INFO:tensorflow:lstm_size: 5
INFO:tensorflow:num_layers: 1
INFO:tensorflow:dropout_rate: 0.3204
INFO:tensorflow:learning_rate: 8.1729e-05
INFO:tensorflow:filter_prob: 0.6827
INFO:tensorflow:job_dir: /tmp/estimator
INFO:tensorflow:model_name: estimator
INFO:tensorflow:verbosity: INFO
INFO:tensorflow:train_steps: 1000.0
INFO:tensorflow:eval_steps: 100.0
INFO:tensorflow:test: False
INFO:tensorflow:keras: False


# Now  Train and Evaluate the Model

In [21]:
import argparse
import io
import json
import os
import pandas as pd
import numpy as np
import sklearn.metrics
#import trainer.model as model
import tensorflow as tf
from google.cloud import storage

In [22]:
# use run experiment contents step by step
SELECT_COLUMN = [19, ] + list(range(21, 29))
hparams = args

select_cols = SELECT_COLUMN
feat_col_names = ['ActivePower_{}'.format(i + 1)
                    for i in range(hparams.seq_len)]


train_input = lambda: make_input_fn( ### delete model.
  data_file=hparams.train_file, ### potential error, if hparams i.e. args does not tranport the file correctly
  seq_len=hparams.seq_len,
  batch_size=hparams.train_batch_size,
  cols=select_cols,
  train_flag=True,
  num_epochs=hparams.num_epochs,
  filter_prob=hparams.filter_prob)

eval_input = lambda: make_input_fn( ### delete model.
  data_file=hparams.eval_file, ### potential error, if hparams i.e. args does not tranport the file correctly
  seq_len=hparams.seq_len,
  batch_size=hparams.eval_batch_size,
  cols=select_cols,
  num_epochs=1)


model_dir = os.path.join(
  hparams.job_dir,
  json.loads(os.environ.get('TF_CONFIG', '{}'))
  .get('task', {}).get('trial', '')
  )


tf.logging.info('model dir {}'.format(model_dir))


  # Experiment running configuration
  # Checkpoint is configured to be saved every ten minutes
run_config = tf.estimator.RunConfig(save_checkpoints_steps=2500)
run_config = run_config.replace(model_dir=model_dir)


params = {'feat_cols': feat_col_names,   ## if using hparams aka args does not work i can specify the dict manually here
            'seq_len': hparams.seq_len,
            'lstm_size': hparams.lstm_size,
            'batch_size': hparams.train_batch_size,
            'num_appliances': len(select_cols) - 1,
            'num_layers': hparams.num_layers,
            'learning_rate': hparams.learning_rate,
            'dropout_rate': hparams.dropout_rate,
            'use_keras': hparams.keras}

estimator = tf.estimator.Estimator(model_fn=model_fn, ### delete model.
                                     model_dir=model_dir,
                                     config=run_config,
                                     params=params)


  # Set training spec
early_stopping = tf.estimator.experimental.stop_if_no_increase_hook(
    estimator,
    metric_name='f_measure',
    max_steps_without_increase=3000,
    min_steps=100,
    run_every_secs=300)


train_spec = tf.estimator.TrainSpec(input_fn=train_input,
                                      max_steps=hparams.train_steps,
                                      hooks=[early_stopping])


# Set serving function, exporter and evaluation spec
# The serving function is only applicable for JSON format input
serving_function = json_serving_input_fn(feat_names=feat_col_names)

exporter = tf.estimator.FinalExporter(name=hparams.model_name,
                                        serving_input_receiver_fn=serving_function)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input,
                                    steps=None,
                                    throttle_secs=120,
                                    exporters=[exporter],
                                    name='energy-disaggregation-eval')

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:model dir /tmp/estimator/
INFO:tensorflow:Using config: {'_model_dir': '/tmp/estimator/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f056371df28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
IN

({'f_measure': 0.018778278,
  'global_step': 1000,
  'loss': 0.374534,
  'precision': 0.0048355046,
  'recall': 0.014594658},
 [b'/tmp/estimator/export/estimator/1613843488'])

In [ ]:
# Run the training job
run_experiment(args)